In [1]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\web\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

#es = Elasticsearch("http://elasticsearch:9200", basic_auth=("elastic", ELASTIC_PASSWORD))
es = Elasticsearch("http://localhost:9200", basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()


ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': 'XjnfooMzSgWFq38Y6psNRA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
# TODO import documents into elastic
import os

# Create index
index_name = "my_index"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

folder_path = "DataAssignment4"

for i, filename in enumerate(os.listdir(folder_path), start=1):
        # Solo file con estensione .txt
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            
            # Legge il contenuto del file
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
            
            # Struttura del documento da importare
            doc = {
                "title": filename,
                "content": content
            }
            
            # Importa il documento in Elasticsearch
            es.index(index=index_name, id=i, document=doc)
            print(f"Document '{filename}' imported succesfully.")


Index 'my_index' already exists.
Document 'Text1.txt' imported succesfully.
Document 'Text2.txt' imported succesfully.
Document 'Text3.txt' imported succesfully.
Document 'Text4.txt' imported succesfully.
Document 'Text5.txt' imported succesfully.
Document 'Text6.txt' imported succesfully.


In [21]:
# TODO Query "money"
response = es.search(index=index_name, query={"match": {"content": "money"}})

print("Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])


Results:
{'title': 'Text2.txt', 'content': 'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What\'s happened to me?" he thought. It wasn\'t a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff tha

In [22]:
# TODO Query "money *"
response = es.search(index=index_name, query={"match": {"content": "money *"}})

print("Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Results:
{'title': 'Text2.txt', 'content': 'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What\'s happened to me?" he thought. It wasn\'t a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff tha

In [25]:
# TODO Query "money for charity"
response = es.search(index=index_name, query={"match_phrase": {"content": "money for charity"}})

print("Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Results:


For the resolution of this query I didn't use the same keyword as before (match) because I wasn't looking for a single word but for a phrase. Here I used match_phrase so I can look for that phrase with words in that specific order.
 
Seems that there's no phrase like that in the documents.

In [26]:
# TODO Query "money for charity" in an alternative way
response = es.search(index=index_name, query={"match": {"content": "money for charity"}})

print("Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Results:
{'title': 'Text2.txt', 'content': 'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What\'s happened to me?" he thought. It wasn\'t a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff tha

This way I used is the wrong way because when we have a phrase, like in this case, we don't have to use only the keyword match. Using match_phrase we'll look for the words in that specific order. Otherwise, using match we got more documents but this documents are returned because they contain each specified word but not in the specified order ("money for charity").